In [2]:
import numpy as np
import laser_hockey_env as lh
import gym
import tensorflow as tf
from importlib import reload

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#np.set_printoptions(suppress=True)
np.set_printoptions(threshold = 1e5)

# Normal Game Play

In [4]:
reload(lh)

<module 'laser_hockey_env' from '/Users/shane/Desktop/Uni/ReinforcementLearning/final/laser-hockey-env-master/laser_hockey_env.py'>

In [8]:
env = lh.LaserHockeyEnv(mode=0)

have a look at the initialization condition: alternating who starts and are random in puck position

In [9]:
obs = env.reset()
#obs_agent2 = env.obs_agent_two()
#env.render()
obs

array([-6.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  6.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.9329834 ,  0.38162279,  0.        ,
        0.        ])

# Model Recovery Test

In [8]:
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('DDPG_model.ckpt.meta')
    saver.restore(sess, 'DDPG_model.ckpt')
    #saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    
    for op in graph.get_operations():
        print(op.name)
    
    #Qs = tf.get_collection('pred_Qs')[0]
    Qs = graph.get_operation_by_name('DQNAgent_2/Q_target/pred').outputs[0]
    #Qs = graph.get_tensor_by_name('DQNAgent/Q_target/dense_1/kernel:0')
    
    #state = tf.get_collection('state')
    state = graph.get_tensor_by_name('DQNAgent_2/Q_target/state:0')
    qs = np.argmax(sess.run(Qs, {state: np.asarray(obs).reshape(-1,16)}))
    print ('decision: ', qs)

INFO:tensorflow:Restoring parameters from DDPG_model.ckpt
DDPGAgent/Q_eval/state
DDPGAgent/Q_eval/action
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform/shape
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform/min
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform/max
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform/RandomUniform
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform/sub
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform/mul
DDPGAgent/Q_eval/w1_s/Initializer/random_uniform
DDPGAgent/Q_eval/w1_s
DDPGAgent/Q_eval/w1_s/Assign
DDPGAgent/Q_eval/w1_s/read
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform/shape
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform/min
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform/max
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform/RandomUniform
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform/sub
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform/mul
DDPGAgent/Q_eval/w1_a/Initializer/random_uniform
DDPGAgent/Q_eval/w1_a
DDPGAgent/Q_eval/w1_a/Assign
DDPGAgen

KeyError: "The name 'DQNAgent_2/Q_target/pred' refers to an Operation not in the graph."

# For DQN Agents

In [7]:
# simulate using DQNAgent
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('DQN_model.ckpt.meta')
    saver.restore(sess, 'DQN_model.ckpt')
    #saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    
    #for op in graph.get_operations():
    #    print(op.name)›
    
    #Qs = tf.get_collection('pred_Qs')[0]
    Qs = graph.get_operation_by_name('DQNAgent/Q_target/pred').outputs[0]
    #Qs = graph.get_tensor_by_name('DQNAgent/Q_target/dense_1/kernel:0')
    
    #state = tf.get_collection('state')
    state = graph.get_tensor_by_name('DQNAgent/Q_target/state:0')
    #state = graph.get_operation_by_name('DQNAgent/Q_target/state').outputs[0]
    
    #obs = np.asarray(obs).reshape(-1, 16)
    #print ('obs: ', obs)
    #qs = np.argmax(sess.run(Qs, {state: np.asarray(obs).reshape(-1,16)}))
    
    obs = env.reset()
    for _ in range(1000):
        env.render()
        qs = np.argmax(sess.run(Qs, {state: np.asarray(obs).reshape(-1,16)}))
        a1 = env.discrete_to_continous_action(qs)
        a2 = np.random.uniform(-1,1,3)    
        obs, r, d, info = env.step(np.hstack([a1,a2]))    
        #print debug info to improve the reward definition
        print (info)
        obs_agent2 = env.obs_agent_two()
        if d: break

    print ('points: ', r)
    env.close()

INFO:tensorflow:Restoring parameters from DQN_model.ckpt
{'winner': 0, 'reward_closeness_to_puck': -0.007012328833933671, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0069625559705168965, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.006888728173346132, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.006791431858854512, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.006671284399226613, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.006528938008673001, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.006365089128446397, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.006229878845026221, 'reward_touch_puck': 0.0, 'rew

{'winner': 0, 'reward_closeness_to_puck': -0.007895915092218379, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.007872182982078844, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.007846922238400118, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.007820813859894898, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.00779477561896163, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.007769953531024387, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.007747705455716549, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.007729588073927596, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_t

{'winner': 0, 'reward_closeness_to_puck': -0.012430071512081234, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012430058914939476, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012430047892445329, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012430038175514597, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012430029245907156, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012430021372704865, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012430014286824794, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01243000798826632, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_t

{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck'

{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.01242996021095751, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck'

{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_

{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.012422134346242886, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_

In [8]:
env.close()

# For DDPG Agents

In [18]:
AGAINST_HUMAN = True
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('DDPG_model.ckpt.meta')
    saver.restore(sess, 'DDPG_model.ckpt')
    #saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    
    playerHuman = lh.HumanOpponent(env=env, player=2)
    
    pred = graph.get_operation_by_name('DDPGAgent/Mu_target/predMu/Tanh').outputs[0]    
    state = graph.get_tensor_by_name('DDPGAgent/Mu_target/state:0')
    
    ####### Test Output ######
    #As = sess.run(pred, {state: np.asarray(obs).reshape(-1,16)})
    #print (As[0])
    
    obs = env.reset()
    reward = 0
    for _ in range(2000):
        env.render()
        As = sess.run(pred, {state: np.asarray(obs).reshape(-1,16)})[0]
        if AGAINST_HUMAN:
            a2 = playerHuman.act(env.obs_agent_two())
        else:
            a2 = np.random.uniform(-1,1,3)    
        obs, r, d, info = env.step(np.hstack([As,a2]))    
        reward += r
        #print (info)
        if d: break

    print ('total points: ', reward)
    env.close()

INFO:tensorflow:Restoring parameters from DDPG_model.ckpt
Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s
Player 1 scored
total points:  29.14172707199081


"info" dict contains useful proxy rewards and winning information

In [8]:
info

{'winner': 0,
 'reward_closeness_to_puck': -0.016178733106732124,
 'reward_touch_puck': 0.0,
 'reward_puck_direction': -5.416407436132432e-06}

Winner == 0: draw

Winner == 1: you (left player)

Winner == -1: opponent wins (right player)

# Hand-crafted Opponent

In [29]:
reload(lh)

<module 'laser_hockey_env' from '/home/shane/Desktop/ReinforcementLearning/final/laser-hockey-env-master/laser_hockey_env.py'>

In [46]:
env = lh.LaserHockeyEnv()

In [47]:
o = env.reset()
env.render()

True

In [48]:
player1 = lh.BasicOpponent()
player2 = lh.BasicOpponent()

In [49]:
obs_buffer = []

In [52]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(600):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_buffer.append(obs)
    obs_agent2 = env.obs_agent_two()
    if d: break

In [45]:
obs_buffer = np.asarray(obs_buffer)

In [46]:
np.mean(obs_buffer,axis=0)

array([-6.21772583,  0.14022074, -0.50388751,  0.06938814,  0.42967239,
       -0.05056231,  5.99394432,  0.190191  , -0.04062059,  1.15586982,
       -0.13772885, -0.0704963 , -0.85041827,  0.12573967, -2.2194066 ,
       -0.30853598])

In [47]:
np.std(obs_buffer,axis=0)

array([ 1.09465005,  1.7363143 ,  2.48617842,  4.01910657,  5.39319311,
        2.70587981,  1.44744218,  1.54582286,  0.24750636,  4.41002617,
        5.06345003,  0.4849688 ,  2.66636978,  2.25679011, 13.08127676,
        7.6629061 ])

In [ ]:
scaling = [ 1.0,  1.0 , 3.14, 4.0, 4.0, 2.0,  
            1.0,  1.0,  3.14, 4.0, 4.0, 2.0,  
            2.0, 2.0, 10.0, 10.0]

# Human Opponent

In [30]:
import time

In [31]:
env = lh.LaserHockeyEnv()

In [32]:
o = env.reset()
env.render()

True

In [33]:
player1 = lh.HumanOpponent(env=env, player=1)
player2 = lh.BasicOpponent()

Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s


In [ ]:
obs = env.reset()
time.sleep(1)
obs_agent2 = env.obs_agent_two()
for _ in range(600):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    print (info)
    if d: break